In [ ]:
# Common imports

import os
import json
import traceback
# import copier as COPY

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager() 
    #fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")


In [ ]:
#import mflib


from mflib.mflib import MFLib

print(f"MFLib version  {MFLib.__version__} " )

slice_name = "MyMonitoredSliceTwo"
print(f"Using slice_name **** {slice_name} ****")


In [ ]:
# SET SLICE INFO

# Sites without PTP + NCSA where meas node is located
#[site1,site2,site3] = fablib.get_random_sites(count=3, avoid=["DALL","GPN","LBNL","RENC","SALT","TACC","UKY","WASH","NCSA","LOSA","GATECH","INDI","NEWY","SRI"])

# Sites without PTP + NCSA where meas node is located + Problem sites
[site1,site2,site3] = fablib.get_random_sites(count=3, avoid=["DALL","GPN","LBNL","RENC","SALT","TACC","UKY","WASH","NCSA","LOSA","GATECH","INDI","MAX", "MASS","NEWY","SRI","UCSD"])


node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'

network1_name='net1'
network2_name='net2'
network3_name='net3'

node1_nic_name = 'nic1'
node2_nic_name = 'nic2'
node3_nic_name = 'nic3'

print(f"Setting up slice {slice_name}")
print(f"Using sites {site1}, {site2}, {site3}")


In [ ]:
# Create topology

try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Node1
    node1 = slice.add_node(name=node1_name, site=site1, image='docker_rocky_8')
    iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
    
    # Node2
    node2 = slice.add_node(name=node2_name, site=site2, image='docker_rocky_8')
    iface2  = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]

    # Node3
    node3 = slice.add_node(name=node3_name, site=site3, image='docker_rocky_8')
    iface3  = node3.add_component(model='NIC_Basic', name=node3_nic_name).get_interfaces()[0]

    # Networks
    net1 = slice.add_l3network(name=network1_name, interfaces=[iface1], type='IPv4')
    net2 = slice.add_l3network(name=network2_name, interfaces=[iface2], type='IPv4')
    net3 = slice.add_l3network(name=network3_name, interfaces=[iface3], type='IPv4')

    print(f"Slice Topology Done.")

    # Add measurement node to topology using static method.
    MFLib.addMeasNode(slice, disk=100, image='docker_ubuntu_20')
    print("Done")

    slice.submit()

    print("✅ DONE! Topology created successfully")
except Exception as e:
    print(f"Exception: {e}")



In [ ]:
# IF need to restart notebook:

#slice = fablib.get_slice(slice_name)
#slice.wait_jupyter(interval=60)

In [ ]:
# Get the assigned subnet:

try:
    network1 = slice.get_network(name=network1_name)
    network1_available_ips = network1.get_available_ips()
    network1.show()
    
    network2 = slice.get_network(name=network2_name)
    network2_available_ips =  network2.get_available_ips()
    network2.show()
    
    network3 = slice.get_network(name=network3_name)
    network3_available_ips =  network3.get_available_ips()
    network3.show()
    
    
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
# Configure nodes:

#NODE 1

try:
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network1_name)  
    node1_addr = network1_available_ips.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=network1.get_subnet())
    
    node1.ip_route_add(subnet=network2.get_subnet(), gateway=network1.get_gateway())
    node1.ip_route_add(subnet=network3.get_subnet(), gateway=network1.get_gateway())
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    stdout, stderr = node1.execute(f'ip route list')

except Exception as e:
    print(f"Exception: {e}")

#Node 2:

try:
    node2 = slice.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network2_name) 
    node2_addr = network2_available_ips.pop(0)
    node2_iface.ip_addr_add(addr=node2_addr, subnet=network2.get_subnet())
    
    node2.ip_route_add(subnet=network1.get_subnet(), gateway=network2.get_gateway())
    node2.ip_route_add(subnet=network3.get_subnet(), gateway=network2.get_gateway())
    
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')    
    stdout, stderr = node2.execute(f'ip route list')

except Exception as e:
    print(f"Exception: {e}")

#Node 3:

try:
    node3 = slice.get_node(name=node3_name)        
    node3_iface = node3.get_interface(network_name=network3_name) 
    node3_addr = network3_available_ips.pop(0)
    node3_iface.ip_addr_add(addr=node3_addr, subnet=network3.get_subnet())
    
    node3.ip_route_add(subnet=network1.get_subnet(), gateway=network3.get_gateway())
    node3.ip_route_add(subnet=network2.get_subnet(), gateway=network3.get_gateway())
    
    stdout, stderr = node3.execute(f'ip addr show {node3_iface.get_os_interface()}')    
    stdout, stderr = node3.execute(f'ip route list')

except Exception as e:
    print(f"Exception: {e}")

In [ ]:
# Instrumentize a slice:

mf = MFLib(slice_name)

In [ ]:
# Install prometheus:
instrumetize_results = mf.instrumentize()

In [ ]:
# Grafana SSH Tunnel Command
# mf.grafana_tunnel_local_port = 10010 # optionally change the port
print(mf.grafana_tunnel)

print(f"Browse to https://localhost:{mf.grafana_tunnel_local_port}/grafana/dashboards?query=%2A")

In [ ]:


# The grafana_manager service was created by the mf.instrumentize call.
# Get access info for Grafana by using the mflib.info call to the grafana_manager.
# Create a dictionary to pass to the service.
data = {}
# Set the info you want to get.
data["get"] = ["admin_password"]
# Call info using service name and data dictionary.
info_results = mf.info("grafana_manager", data)
print(info_results)



In [ ]:
# UNCOMMENT the line below and execute this cell, then try again. 
#slice.delete()